# module

In [ ]:
import sys, os
sys.path.append(os.pardir)
sys.path.append('../')
sys.path.append('./')
import numpy as np
import matplotlib.pyplot as plt

import cv2
import skimage.transform as sktrans
import skimage.filters as skfilt
import tensorflow.keras.layers
from tensorflow.keras.layers import Dense, Input, Activation, Flatten, Lambda
from tensorflow.keras.layers import BatchNormalization,Add,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LeakyReLU, ReLU, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, UpSampling2D, concatenate,Activation
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from S1_data import *            # 이 안에 있는 라이브러리들은 조금 수정이 필요합니다
import time
T1 = time.time()
my_path = os.getcwd()

2023-12-12 04:48:07.897099: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 04:48:08.057194: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
#from osgeo import gdal
import sys, os
sys.path.append(os.pardir)
sys.path.append('../../')


import pandas as pd
from sklearn.impute import KNNImputer
from skimage import io, util
import shutil
import pandas as pd
from tqdm import tqdm
import random

In [ ]:
def read_binary(filename,naz,nrg,nband=1,data_type='f'):
    """ read binary data
    """
    fd = open(filename,'rb')
    bin_data=np.fromfile(fd,dtype =np.dtype(data_type),count=naz*nrg*nband)
    if nband == 1:
        return bin_data.reshape(naz,nrg)
    else: 
        return bin_data.reshape(nband,naz,nrg)

def write_binary(filename, bin_data):
    """ write binary data
    """    
    fd = open(filename,'wb')
    bin_data.tofile(fd)
    fd.close()

In [9]:

Seoul1_N_HH= 10240 # column
Seoul1_M= 22000 # row

Seoul2_N_HH= 10240 # column
Seoul2_M_HH= 22000 # row


B=1# band number
Bimage=2

## Seoul Flood

In [ ]:
from PIL import Image
Seoul1_files =  [file for file in os.listdir('./labels/Seoul/HH/') if file.endswith('.tif')]
Seoul2_files =  [file for file in os.listdir('./labels/Seoul2/HH/') if file.endswith('.tif')]

Seoul_dataFrame_orig_HH = np.zeros((int(len(Seoul_HH_files)),B,Seoul_N_HH,Seoul_M_HH),'float32')
Seoul_labelFrame_orig1 = np.zeros((int(len(Seoul_HH_files)),Seoul_N_HH,Seoul_M_HH),'float32')

Seoul2_dataFrame_orig_HH = np.zeros((int(len(Seoul_HH_files)),B,Seoul2_N_HH,Seoul2_M_HH),'float32')
Seoul2_labelFrame_orig2 = np.zeros((int(len(Seoul_HH_files)),Seoul2_N_HH,Seoul2_M_HH),'float32')



i1=0
i2=0


HH_list=[]
HH2_list=[]

for i, name in enumerate(Seoul1_files):

    Seoul_dataFrame_orig_HH[i1,:,:,:]=read_binary('./images/Seoul/HH/' + 'Seoul_'+ str(name[8:16])+'_HH',Seoul_N_HH,Seoul_M_HH,B,data_type='float32')
    Seoul_labelFrame_orig1[i1,:,:]=io.imread('./labels/Seoul/HH/' + 'Seoul_'+ str(name[8:16])+'_HH.tif')
    HH_list.append('Seoul_' + name[8:16] + '_HH')
    i1=i1+1


   
for i, name in enumerate(Seoul2_files):

    Seoul2_dataFrame_orig_HH[i1,:,:,:]=read_binary('./images/Seoul2/HH/' + 'Seoul_'+ str(name[8:16])+'_HH',Seoul2_N_HH,Seoul2_M_HH,B,data_type='float32')
    Seoul2_labelFrame_orig2[i1,:,:]=io.imread('./labels/Seoul2/HH/' + 'Seoul_'+ str(name[8:16])+'_HH.tif')
    HH2_list.append('Seoul2_' + name[8:16] + '_HH')
    i1=i1+1
     
Seoul_dataFrame_orig_HH= Seoul_dataFrame_orig_HH.transpose(0,2,3,1) 
Seoul2_dataFrame_orig_HH= Seoul2_dataFrame_orig_HH.transpose(0,2,3,1) 





In [ ]:
Seoul_max0_iw1,Seoul_min0_iw1 = image_norm_pre(Seoul_dataFrame_orig_HH)
Seoul_max0_iw2,Seoul_min0_iw2 = image_norm_pre(Seoul2_dataFrame_orig_HH)

print(Seoul_max0_iw1,Seoul_min0_iw1)
print(Seoul_max0_iw2,Seoul_min0_iw2)

## test scene no normalization

for i in range(Seoul_dataFrame_orig_HH.shape[0]):
    Seoul_dataFrame_orig_HH[i] = image_norm_ALL(Seoul_dataFrame_orig_HH[i],Seoul_max0_iw1,Seoul_min0_iw1)

for i in range(Seoul2_dataFrame_orig_HH.shape[0]):
    Seoul2_dataFrame_orig_HH[i] = image_norm_ALL(Seoul2_dataFrame_orig_HH[i],Seoul_max0_iw2,Seoul_min0_iw2)


    

In [36]:
# Random Shuffle
Seoul_dataFrame_orig_HH,Seoul_labelFrame_orig1,HH_list = random_sf(Seoul_dataFrame_orig_HH,Seoul_labelFrame_orig1,HH_list)
Seoul2_dataFrame_orig_HH,Seoul2_labelFrame_orig2,HH2_list = random_sf(Seoul2_dataFrame_orig_HH,Seoul2_labelFrame_orig2,HH2_list)


In [ ]:
# Train Test Split
Seoul_train,Seoul_label_train,Seoul_test,Seoul_label_test,Seoul_list_train,Seoul_list_test=train_test_split(Seoul_dataFrame_orig_HH,Seoul_labelFrame_orig1,HH_list)
Seoul2_train,Seoul2_label_train,Seoul2_test,Seoul2_label_test,Seoul2_list_train,Seoul2_list_test=train_test_split(Seoul2_dataFrame_orig_HH,Seoul2_labelFrame_orig2,HH2_list)


### crop image

In [12]:
## train patch

Seoul_train_patch,Seoul_label_train_patch,Seoul_train_idx = crop_image_shift(Seoul_train,Seoul_label_train,patch_size=(512,512),max_shift=32)
Seoul2_train_patch,Seoul2_label_train_patch,Seoul2_train_idx = crop_image_shift(Seoul2_train,Seoul2_label_train,patch_size=(512,512),max_shift=32)

In [ ]:
#Patch Selected
Seoul_train_patch_sel,Seoul_label_train_patch_sel = select_patch(Seoul_train_patch,Seoul_label_train_patch)
Seoul2_train_patch_sel,Seoul2_label_train_patch_sel = select_patch(Seoul2_train_patch,Seoul2_label_train_patch)


In [ ]:
Seoul_train_patch_final = np.vstack([Seoul_train_patch_sel,Seoul2_train_patch_sel])
Seoul_train_label_patch_final = np.vstack([Seoul_label_train_patch_sel,Seoul2_label_train_patch_sel])



In [18]:
tmp1=np.arange(Seoul_train_patch_final.shape[0])
np.random.seed(200)
np.random.shuffle(tmp1)
Seoul_train_label_patch_final=Seoul_train_label_patch_final[tmp1,:,:]
Seoul_train_patch_final=Seoul_train_patch_final[tmp1,:,:,:]
print(f"Number of train Data: {Seoul_train_patch_final.shape[0]}")

import pickle
with open('./pickle/Seoul_train_patch_512_100.pkl', 'wb') as f:
	pickle.dump(Seoul_train_patch_final, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('./pickle/Seoul_train_label_patch_512_100.pkl', 'wb') as f:
	pickle.dump(Seoul_train_label_patch_final, f, protocol=pickle.HIGHEST_PROTOCOL)

Number of train Data: 3000


### crop test

In [20]:
## test patch

Seoul_test_patch,Seoul_label_test_patch,Seoul_test_idx = crop_image_shift(Seoul_test,Seoul_label_test,patch_size=(512,512),max_shift=32)
Seoul2_test_patch,Seoul2_label_test_patch,Seoul2_test_idx = crop_image_shift(Seoul2_test,Seoul2_label_test,patch_size=(512,512),max_shift=32)

In [ ]:
#Patch Selected
Seoul_test_patch_sel,Seoul_label_test_patch_sel = select_patch(Seoul_test_patch,Seoul_label_test_patch)
Seoul2_test_patch_sel,Seoul2_label_test_patch_sel = select_patch(Seoul2_test_patch,Seoul2_label_test_patch)


In [ ]:
Seoul_test_patch_final = np.vstack([Seoul_test_patch_sel,Seoul2_test_patch_sel])
Seoul_test_label_patch_final = np.vstack([Seoul_label_test_patch_sel,Seoul2_label_test_patch_sel])



In [23]:
tmp1=np.arange(Seoul_test_patch_final.shape[0])
np.random.seed(200)
np.random.shuffle(tmp1)
Seoul_test_label_patch_final=Seoul_test_label_patch_final[tmp1,:,:]
Seoul_test_patch_final=Seoul_test_patch_final[tmp1,:,:,:]
print(f"Number of test Data: {Seoul_test_patch_final.shape[0]}")

import pickle
with open('./pickle/Seoul_test_patch_512_100.pkl', 'wb') as f:
	pickle.dump(Seoul_test_patch_final, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('./pickle/Seoul_test_label_patch_512_100.pkl', 'wb') as f:
	pickle.dump(Seoul_test_label_patch_final, f, protocol=pickle.HIGHEST_PROTOCOL)

Number of train Data: 180
